# Synthetic microbial consortium by *Chen et al. 2015*

From:

Chen Y, Kim JK, Hirning AJ, Josić K, Bennett MR. SYNTHETIC BIOLOGY. Emergent genetic oscillations in a synthetic microbial consortium. Science. 2015;349(6251):986-989. doi:10.1126/science.aaa3794 

In this oscilator consortium, there are:
- activator strain, which produces rhlI(C4), which causes CFP production and self-upregulates. When it senses C14, it produces AiiA which degrades both signaling molecules, and LacI which represesses both rhlI(C4) and CFP production. 
- repressor strain, which produces cinI(C14) in response to C4. C14 causes production of YFP, LacI (which represses YFP and cinI(C14) production) and AiiA, which degrades both signalling molecules.

First, install the latest version of LOICA Consortia.

In [ ]:
pip install -e .

In [ ]:
import loica as lc
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Then add gene products involved in genetic networks. 

Since AiiA acts as a repressor by degradation of AHL and current LOICA does not have a class for degradation enzymes, this code is not valid yet.

## P2N2

In [ ]:
activator = lc.GeneticNetwork()

# TODO: determine degr rate
rhli = lc.Regulator(name='RhlI')
c4 = lc.Regulator(name='C4', diffusion_rate=1)
aiia = lc.Regulator(name="AiiA", degradation_rate=0)
laci = lc.Regulator(name='LacI', degradation_rate=1)
cfp = lc.Reporter(name="CFP", degradation_rate=1, color='green')
c14 = lc.Regulator(name='C14', diffusion_rate=1)

activator.add_regulator([rhli, c4, aiia, laci, c14])
activator.add_reporter(cfp)

# TODO: find a, K and n
c14_aiia = lc.Hill1(c14, aiia, a=[1, 100], K=10, n=2, name='Pcin')
c14_laci = lc.Hill1(c14, laci, a=[1, 100], K=10, n=2, name='Pcin')

aiia_deg_c14 = lc.Degrader(enzyme=aiia, substrate=c14, k1=1, k1r=1, k2=3)
aiia_deg_c4 = lc.Degrader(enzyme=aiia, substrate=c4, k1=1, k1r=1, k2=3)
# or
aiia_deg_ahls = lc.Degrader(enzyme=aiia, substrate=[c14, c4], k1=1, k1r=1, k2=3)

c4_laci_cfp = lc.Hill2(input=[c4, laci], output=cfp, alpha=[0.1, 100, 0.1, 0.1], K=[10, 10], n=[2, 2], name='Prhl/lac')
c4_laci_rhli = lc.Hill2(input=[c4, laci], output=rhli, alpha=[0.1, 100, 0.1, 0.1], K=[10, 10], n=[2, 2], name='Prhl/lac')

rhli_c4 = lc.Producer(enzyme=rhli, product=c4, rate=1)

activator.add_operator([c14_aiia, c14_laci, aiia_deg_c14, aiia_deg_c4, c4_laci_cfp, c4_laci_rhli, rhli_c4])

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=200)
activator.draw(pos=nx.planar_layout)

Next, add circuit to strain with defined metabolism

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 0.5)

def biomass(t):
    return lc.gompertz(t, 0.01, 1, 1, 0.5)

metab_act = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
activator_strain = lc.Strain(name="Activator", genetic_network=activator, metabolism=metab_act)

In [ ]:
repressor = lc.GeneticNetwork()

# TODO: determine degr rate
cini = lc.Regulator(name='CinI')
c4_r = lc.Regulator(name='C4', diffusion_rate=1)
aiia_r = lc.Regulator(name="AiiA", degradation_rate=0)
laci_r = lc.Regulator(name='LacI', degradation_rate=1)
yfp = lc.Reporter(name="YFP", degradation_rate=1, color='green')
c14_r = lc.Regulator(name='C14', diffusion_rate=1)

repressor.add_regulator([cini, c4_r, aiia_r, laci_r, c14_r])
repressor.add_reporter(yfp)

# TODO: find a, K and n
c14_aiia_r = lc.Hill1(c14_r, aiia_r, a=[1, 100], K=10, n=2, name='Pcin')
c14_laci_r = lc.Hill1(c14_r, laci_r, a=[1, 100], K=10, n=2, name='Pcin')

aiia_deg_c14_r = lc.Degrader(enzyme=aiia_r, substrate=c14_r, k1=1, k1r=1, k2=3)
aiia_deg_c4_r = lc.Degrader(enzyme=aiia_r, substrate=c4_r, k1=1, k1r=1, k2=3)
# or
aiia_deg_ahls_r = lc.Degrader(enzyme=aiia_r, substrate=[c14_r, c4_r], k1=1, k1r=1, k2=3)

c14_laci_yfp = lc.Hill2(input=[c14_r, laci_r], output=yfp, alpha=[0.1, 100, 0.1, 0.1], K=[10, 10], n=[2, 2], name='Pcin/lac')
c14_laci_cini = lc.Hill2(input=[c14_r, laci_r], output=cini, alpha=[0.1, 100, 0.1, 0.1], K=[10, 10], n=[2, 2], name='Pcin/lac')

cini_c14 = lc.Producer(enzyme=cini, product=c14_r, rate=1)

repressor.add_operator([c14_aiia_r, c14_laci_r, aiia_deg_c14_r, aiia_deg_c4_r, c14_laci_yfp, c14_laci_cini, cini_c14])

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=200)
repressor.draw(pos=nx.planar_layout)

Next, add circuit to strain with defined metabolism

In [ ]:
metab_rep = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
repressor_strain = lc.Strain(name="Repressor", genetic_network=repressor, metabolism=metab_rep)

In [ ]:
sample = lc.Sample(strain=[activator_strain, repressor_strain])
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='Quorum clock',
                description='Simulated GFP expression of quorum clock by Danino et al. (2010)'
                )

In [ ]:
assay.run(stochastic=False, track_all=True)

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
# TODO: rewrite
# m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='AHL in E. coli'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='yemGFP in E. coli'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='AiiA in E. coli'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='E. coli biomass'].plot(x='Time', y='Measurement', style='-', ax=ax)
# plt.legend(['Extracellular AHL', 'Internal AHL', 'GFP', 'AiiA', 'Biomass'])

## P1N2

In [ ]:
activator = lc.GeneticNetwork()

# TODO: determine degr rate
rhli = lc.Regulator(name='RhlI')
c4 = lc.Regulator(name='C4', diffusion_rate=1)
aiia = lc.Regulator(name="AiiA", degradation_rate=0)
laci = lc.Regulator(name='LacI', degradation_rate=1)
cfp = lc.Reporter(name="CFP", degradation_rate=1, color='green')
c14 = lc.Regulator(name='C14', diffusion_rate=1)

activator.add_regulator([rhli, c4, aiia, laci, c14])
activator.add_reporter(cfp)

# TODO: find a, K and n
c14_aiia = lc.Hill1(c14, aiia, a=[1, 100], K=10, n=2, name='Pcin')
c14_laci = lc.Hill1(c14, laci, a=[1, 100], K=10, n=2, name='Pcin')

aiia_deg_c14 = lc.Degrader(enzyme=aiia, substrate=c14, k1=1, k1r=1, k2=3)
aiia_deg_c4 = lc.Degrader(enzyme=aiia, substrate=c4, k1=1, k1r=1, k2=3)
# or
aiia_deg_ahls = lc.Degrader(enzyme=aiia, substrate=[c14, c4], k1=1, k1r=1, k2=3)

laci_cfp = lc.Hill1(laci, cfp, a=[100, 1], K=10, n=2, name='Plac')
laci_rhli = lc.Hill1(laci, rhli, a=[100, 1], K=10, n=2, name='Plac')

rhli_c4 = lc.Producer(enzyme=rhli, product=c4, rate=1)

activator.add_operator([c14_aiia, c14_laci, aiia_deg_c14, aiia_deg_c4, laci_cfp, laci_rhli, rhli_c4])

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=200)
activator.draw(pos=nx.planar_layout)

Next, add circuit to strain with defined metabolism

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 0.5)

def biomass(t):
    return lc.gompertz(t, 0.01, 1, 1, 0.5)

metab_act = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
activator_strain = lc.Strain(name="Activator", genetic_network=activator, metabolism=metab_act)

In [ ]:
repressor = lc.GeneticNetwork()

# TODO: determine degr rate
cini = lc.Regulator(name='CinI')
c4_r = lc.Regulator(name='C4', diffusion_rate=1)
aiia_r = lc.Regulator(name="AiiA", degradation_rate=0)
laci_r = lc.Regulator(name='LacI', degradation_rate=1)
yfp = lc.Reporter(name="YFP", degradation_rate=1, color='green')
c14_r = lc.Regulator(name='C14', diffusion_rate=1)

repressor.add_regulator([cini, c4_r, aiia_r, laci_r, c14_r])
repressor.add_reporter(yfp)

# TODO: find a, K and n
c14_aiia_r = lc.Hill1(c14_r, aiia_r, a=[1, 100], K=10, n=2, name='Pcin')
c14_laci_r = lc.Hill1(c14_r, laci_r, a=[1, 100], K=10, n=2, name='Pcin')

aiia_deg_c14_r = lc.Degrader(enzyme=aiia_r, substrate=c14_r, k1=1, k1r=1, k2=3)
aiia_deg_c4_r = lc.Degrader(enzyme=aiia_r, substrate=c4_r, k1=1, k1r=1, k2=3)
# or
aiia_deg_ahls_r = lc.Degrader(enzyme=aiia_r, substrate=[c14_r, c4_r], k1=1, k1r=1, k2=3)

c14_laci_yfp = lc.Hill2(input=[c14_r, laci_r], output=yfp, alpha=[0.1, 100, 0.1, 0.1], K=[10, 10], n=[2, 2], name='Pcin/lac')
c14_laci_cini = lc.Hill2(input=[c14_r, laci_r], output=cini, alpha=[0.1, 100, 0.1, 0.1], K=[10, 10], n=[2, 2], name='Pcin/lac')

cini_c14 = lc.Producer(enzyme=cini, product=c14_r, rate=1)

repressor.add_operator([c14_aiia_r, c14_laci_r, aiia_deg_c14_r, aiia_deg_c4_r, c14_laci_yfp, c14_laci_cini, cini_c14])

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=200)
repressor.draw(pos=nx.planar_layout)

Next, add circuit to strain with defined metabolism

In [ ]:
metab_rep = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
repressor_strain = lc.Strain(name="Repressor", genetic_network=repressor, metabolism=metab_rep)

In [ ]:
sample = lc.Sample(strain=[activator_strain, repressor_strain])
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='Quorum clock',
                description='Simulated GFP expression of quorum clock by Danino et al. (2010)'
                )

In [ ]:
assay.run(stochastic=False, track_all=True)

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
# TODO: rewrite
# m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='AHL in E. coli'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='yemGFP in E. coli'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='AiiA in E. coli'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='E. coli biomass'].plot(x='Time', y='Measurement', style='-', ax=ax)
# plt.legend(['Extracellular AHL', 'Internal AHL', 'GFP', 'AiiA', 'Biomass'])

## P2N1

In [ ]:
activator = lc.GeneticNetwork()

# TODO: determine degr rate
rhli = lc.Regulator(name='RhlI')
c4 = lc.Regulator(name='C4', diffusion_rate=1)
aiia = lc.Regulator(name="AiiA", degradation_rate=0)
laci = lc.Regulator(name='LacI', degradation_rate=1)
cfp = lc.Reporter(name="CFP", degradation_rate=1, color='green')
c14 = lc.Regulator(name='C14', diffusion_rate=1)

activator.add_regulator([rhli, c4, aiia, laci, c14])
activator.add_reporter(cfp)

# TODO: find a, K and n
c14_aiia = lc.Hill1(c14, aiia, a=[1, 100], K=10, n=2, name='Pcin')
c14_laci = lc.Hill1(c14, laci, a=[1, 100], K=10, n=2, name='Pcin')

aiia_deg_c14 = lc.Degrader(enzyme=aiia, substrate=c14, k1=1, k1r=1, k2=3)
aiia_deg_c4 = lc.Degrader(enzyme=aiia, substrate=c4, k1=1, k1r=1, k2=3)
# or
aiia_deg_ahls = lc.Degrader(enzyme=aiia, substrate=[c14, c4], k1=1, k1r=1, k2=3)

c4_laci_cfp = lc.Hill2(input=[c4, laci], output=cfp, alpha=[0.1, 100, 0.1, 0.1], K=[10, 10], n=[2, 2], name='Prhl/lac')
c4_laci_rhli = lc.Hill2(input=[c4, laci], output=rhli, alpha=[0.1, 100, 0.1, 0.1], K=[10, 10], n=[2, 2], name='Prhl/lac')

rhli_c4 = lc.Producer(enzyme=rhli, product=c4, rate=1)

activator.add_operator([c14_aiia, c14_laci, aiia_deg_c14, aiia_deg_c4, c4_laci_cfp, c4_laci_rhli, rhli_c4])

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=200)
activator.draw(pos=nx.planar_layout)

Next, add circuit to strain with defined metabolism

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 0.5)

def biomass(t):
    return lc.gompertz(t, 0.01, 1, 1, 0.5)

metab_act = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
activator_strain = lc.Strain(name="Activator", genetic_network=activator, metabolism=metab_act)

In [ ]:
repressor = lc.GeneticNetwork()

# TODO: determine degr rate
cini = lc.Regulator(name='CinI')
c4_r = lc.Regulator(name='C4', diffusion_rate=1)
aiia_r = lc.Regulator(name="AiiA", degradation_rate=0)
yfp = lc.Reporter(name="YFP", degradation_rate=1, color='green')
c14_r = lc.Regulator(name='C14', diffusion_rate=1)

repressor.add_regulator([cini, c4_r, aiia_r, c14_r])
repressor.add_reporter(yfp)

# TODO: find a, K and n
c14_aiia_r = lc.Hill1(c14_r, aiia_r, a=[1, 100], K=10, n=2, name='Pcin')

aiia_deg_c14_r = lc.Degrader(enzyme=aiia_r, substrate=c14_r, k1=1, k1r=1, k2=3)
aiia_deg_c4_r = lc.Degrader(enzyme=aiia_r, substrate=c4_r, k1=1, k1r=1, k2=3)
# or
aiia_deg_ahls_r = lc.Degrader(enzyme=aiia_r, substrate=[c14_r, c4_r], k1=1, k1r=1, k2=3)

c14_yfp = lc.Hill1(c14_r, yfp, a=[1, 100], K=10, n=2, name='Pcin/lac')
c4_cini = lc.Hill1(c4_r, cini, a=[1, 100], K=10, n=2, name='Prhl/lac')

cini_c14 = lc.Producer(enzyme=cini, product=c14_r, rate=1)

repressor.add_operator([c14_aiia_r, aiia_deg_c14_r, aiia_deg_c4_r, c14_yfp, c4_cini, cini_c14])

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=200)
repressor.draw(pos=nx.planar_layout)

Next, add circuit to strain with defined metabolism

In [ ]:
metab_rep = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
repressor_strain = lc.Strain(name="Repressor", genetic_network=repressor, metabolism=metab_rep)

In [ ]:
sample = lc.Sample(strain=[activator_strain, repressor_strain])
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='Quorum clock',
                description='Simulated GFP expression of quorum clock by Danino et al. (2010)'
                )

In [ ]:
assay.run(stochastic=False, track_all=True)

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
# TODO: rewrite
# m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='AHL in E. coli'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='yemGFP in E. coli'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='AiiA in E. coli'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='E. coli biomass'].plot(x='Time', y='Measurement', style='-', ax=ax)
# plt.legend(['Extracellular AHL', 'Internal AHL', 'GFP', 'AiiA', 'Biomass'])

## P1N1

In [ ]:
activator = lc.GeneticNetwork()

# TODO: determine degr rate
rhli = lc.Regulator(name='RhlI')
c4 = lc.Regulator(name='C4', diffusion_rate=1)
aiia = lc.Regulator(name="AiiA", degradation_rate=0)
laci = lc.Regulator(name='LacI', degradation_rate=1)
cfp = lc.Reporter(name="CFP", degradation_rate=1, color='green')
c14 = lc.Regulator(name='C14', diffusion_rate=1)

activator.add_regulator([rhli, c4, aiia, laci, c14])
activator.add_reporter(cfp)

# TODO: find a, K and n
c14_aiia = lc.Hill1(c14, aiia, a=[1, 100], K=10, n=2, name='Pcin')
c14_laci = lc.Hill1(c14, laci, a=[1, 100], K=10, n=2, name='Pcin')

aiia_deg_c14 = lc.Degrader(enzyme=aiia, substrate=c14, k1=1, k1r=1, k2=3)
aiia_deg_c4 = lc.Degrader(enzyme=aiia, substrate=c4, k1=1, k1r=1, k2=3)
# or
aiia_deg_ahls = lc.Degrader(enzyme=aiia, substrate=[c14, c4], k1=1, k1r=1, k2=3)

laci_cfp = lc.Hill1(laci, cfp, a=[100, 1], K=10, n=2, name='Plac')
laci_rhli = lc.Hill1(laci, rhli, a=[100, 1], K=10, n=2, name='Plac')

rhli_c4 = lc.Producer(enzyme=rhli, product=c4, rate=1)

activator.add_operator([c14_aiia, c14_laci, aiia_deg_c14, aiia_deg_c4, laci_cfp, laci_rhli, rhli_c4])

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=200)
activator.draw(pos=nx.planar_layout)

Next, add circuit to strain with defined metabolism

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 0.5)

def biomass(t):
    return lc.gompertz(t, 0.01, 1, 1, 0.5)

metab_act = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
activator_strain = lc.Strain(name="Activator", genetic_network=activator, metabolism=metab_act)

In [ ]:
repressor = lc.GeneticNetwork()

# TODO: determine degr rate
cini = lc.Regulator(name='CinI')
c4_r = lc.Regulator(name='C4', diffusion_rate=1)
aiia_r = lc.Regulator(name="AiiA", degradation_rate=0)
yfp = lc.Reporter(name="YFP", degradation_rate=1, color='green')
c14_r = lc.Regulator(name='C14', diffusion_rate=1)

repressor.add_regulator([cini, c4_r, aiia_r, c14_r])
repressor.add_reporter(yfp)

# TODO: find a, K and n
c14_aiia_r = lc.Hill1(c14_r, aiia_r, a=[1, 100], K=10, n=2, name='Pcin')

aiia_deg_c14_r = lc.Degrader(enzyme=aiia_r, substrate=c14_r, k1=1, k1r=1, k2=3)
aiia_deg_c4_r = lc.Degrader(enzyme=aiia_r, substrate=c4_r, k1=1, k1r=1, k2=3)
# or
aiia_deg_ahls_r = lc.Degrader(enzyme=aiia_r, substrate=[c14_r, c4_r], k1=1, k1r=1, k2=3)

c14_yfp = lc.Hill1(c14_r, yfp, a=[1, 100], K=10, n=2, name='Pcin/lac')
c4_cini = lc.Hill1(c4_r, cini, a=[1, 100], K=10, n=2, name='Prhl/lac')

cini_c14 = lc.Producer(enzyme=cini, product=c14_r, rate=1)

repressor.add_operator([c14_aiia_r, aiia_deg_c14_r, aiia_deg_c4_r, c14_yfp, c4_cini, cini_c14])

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=200)
repressor.draw(pos=nx.planar_layout)

Next, add circuit to strain with defined metabolism

In [ ]:
metab_rep = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
repressor_strain = lc.Strain(name="Repressor", genetic_network=repressor, metabolism=metab_rep)

In [ ]:
sample = lc.Sample(strain=[activator_strain, repressor_strain])
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='Quorum clock',
                description='Simulated GFP expression of quorum clock by Danino et al. (2010)'
                )

In [ ]:
assay.run(stochastic=False, track_all=True)

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
# TODO: rewrite
# m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='AHL in E. coli'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='yemGFP in E. coli'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='AiiA in E. coli'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='E. coli biomass'].plot(x='Time', y='Measurement', style='-', ax=ax)
# plt.legend(['Extracellular AHL', 'Internal AHL', 'GFP', 'AiiA', 'Biomass'])